In [ ]:
from ase.io import read, write
from ase.visualize import view
from ase.build import surface
import os

crystals = ['TiN', 'VN', 'ScN', 'ZrN', 'NbN']
crystal_dir = '../crystal/eos_structures'  # Relative path from slab dir

for name in crystals:
    file_path = os.path.join(crystal_dir, f"{name}_D3_EOS_sorted.xyz")
    bulk = read(file_path)

    #view(bulk)
    print(f"Loaded and visualized bulk for {name} from {file_path}")

    # Create a (100) slab with 4 layers and 10 Å vacuum
    slab = surface(bulk, (1, 0, 0), layers=2, vacuum=5.5)
    slab = slab*(2,2,1)
    slab.center(axis=2)

    view(slab)
    print(f"Visualized {name} (100) slab — 4 layers + 10 Å vacuum")


Loaded and visualized bulk for TiN from ../crystal/eos_structures/TiN_D3_EOS_sorted.xyz
Visualized TiN (100) slab — 4 layers + 10 Å vacuum
Loaded and visualized bulk for VN from ../crystal/eos_structures/VN_D3_EOS_sorted.xyz
Visualized VN (100) slab — 4 layers + 10 Å vacuum
Loaded and visualized bulk for ScN from ../crystal/eos_structures/ScN_D3_EOS_sorted.xyz
Visualized ScN (100) slab — 4 layers + 10 Å vacuum
Loaded and visualized bulk for ZrN from ../crystal/eos_structures/ZrN_D3_EOS_sorted.xyz
Visualized ZrN (100) slab — 4 layers + 10 Å vacuum
Loaded and visualized bulk for NbN from ../crystal/eos_structures/NbN_D3_EOS_sorted.xyz
Visualized NbN (100) slab — 4 layers + 10 Å vacuum


In [6]:
from ase.io import read
from ase.visualize import view
import os

crystal_dir = '../crystal/eos_structures'
file_path = os.path.join(crystal_dir, 'TiN_D3_EOS.xyz')
view(read(file_path))

<Popen: returncode: None args: ['/home/ameerracle/miniconda3/envs/gpaw/bin/p...>

In [ ]:
from ase.io import read, write
from ase.build import surface
from ase.visualize import view
import os

crystal_dir = '../crystal/eos_structures'  # Relative path from slab dir
all_xyz_files = [f for f in os.listdir(crystal_dir) if f.endswith('_EOS_sorted.xyz')]

for fname in all_xyz_files:
    file_path = os.path.join(crystal_dir, fname)
    try:
        bulk = read(file_path)

        # Clean the functional name
        name = fname.replace('_EOS_sorted.xyz', '')
        if 'PBE+D3' in name:
            name = name.replace('PBE+D3', 'D3')
        elif 'PBE+D4' in name:
            name = name.replace('PBE+D4', 'D4')
        elif 'BEEF-vdW' in name:
            name = name.replace('BEEF-vdW', 'BEEF')

        # Build a (100) slab: 4 layers, 5.5 Å vacuum, 2×2 supercell
        slab = surface(bulk, (1, 0, 0), layers=2, vacuum=5.5)
        slab.center(axis=2)
        slab = slab * (2, 2, 1)

        # Optional view
        #view(slab)
        print(f"✓ {name}: slab generated and visualized.")

        # Save slab
        write(f"{name}_slab_100.xyz", slab)
    except Exception as e:
        print(f"⚠ Failed to process {fname}: {e}")


✓ ZrN_D3: slab generated and visualized.
✓ NbN_D4: slab generated and visualized.
✓ TiN_BEEF: slab generated and visualized.
✓ VN_D4: slab generated and visualized.
✓ VN_D3: slab generated and visualized.
✓ TiN_D4: slab generated and visualized.
✓ ScN_BEEF: slab generated and visualized.
✓ NbN_BEEF: slab generated and visualized.
✓ VN_BEEF: slab generated and visualized.
✓ ScN_D4: slab generated and visualized.
✓ NbN_D3: slab generated and visualized.
✓ ZrN_BEEF: slab generated and visualized.
✓ TiN_D3: slab generated and visualized.
✓ ScN_D3: slab generated and visualized.
✓ ZrN_D4: slab generated and visualized.


In [ ]:
from ase.io import read, write
from ase.constraints import FixAtoms
from ase.optimize import BFGS
from ase.calculators.mixing import SumCalculator
from dftd3.ase import DFTD3
from dftd4.ase import DFTD4
from gpaw import GPAW, PW, FermiDirac
import os

ecut = 500
kpts = (4, 4, 1)
smearing = 0.05
fmax = 0.02  # force threshold

# All slabs expected in format: 'TiN_D3_slab_100.xyz'
slab_files = [f for f in os.listdir() if f.endswith('D3_slab_100.xyz')]

def get_calculator(name, functional):
    txt_file = f'{name}_{functional}_slab_relax.txt'
    if functional == 'D3':
        return SumCalculator([
            GPAW(mode=PW(ecut), xc='PBE', kpts=kpts,
                 occupations=FermiDirac(smearing), txt=txt_file),
            DFTD3(method="PBE", damping="d3bj")
        ])
    else:
        raise ValueError(f"Unsupported functional in file: {functional}")

for fname in slab_files:
    atoms = read(fname)
    basename = fname.replace('_slab_100.xyz', '')  # e.g., TiN_D3
    name_parts = basename.split('_')
    material = name_parts[0]
    functional = name_parts[1]  # D3, D4, or BEEF

    # Fix bottom 2 layers by z-coordinate
    z_positions = [atom.position[2] for atom in atoms]
    sorted_indices = sorted(range(len(atoms)), key=lambda i: z_positions[i])
    num_fixed = len(atoms) // 2  # fix bottom half (usually 2/4 layers)
    fix_indices = sorted_indices[:num_fixed]
    atoms.set_constraint(FixAtoms(indices=fix_indices))

    # Set calculator
    atoms.calc = get_calculator(material, functional)
    view(atoms)
'''
    # Run relaxation
    traj_file = fname.replace('.xyz', '_relax.traj')
    dyn = BFGS(atoms, trajectory=traj_file)
    dyn.run(fmax=fmax)

    # Save relaxed structure
    relaxed_file = fname.replace('.xyz', '_relaxed.xyz')
    write(relaxed_file, atoms)
    print(f"✔ Relaxed: {relaxed_file}")
'''

'\n    # Run relaxation\n    traj_file = fname.replace(\'.xyz\', \'_relax.traj\')\n    dyn = BFGS(atoms, trajectory=traj_file)\n    dyn.run(fmax=fmax)\n\n    # Save relaxed structure\n    relaxed_file = fname.replace(\'.xyz\', \'_relaxed.xyz\')\n    write(relaxed_file, atoms)\n    print(f"✔ Relaxed: {relaxed_file}")\n'

In [8]:
from ase.io import read
from ase.visualize import view

view(read('TiN_BEEF_slab_100.xyz'))

<Popen: returncode: None args: ['/home/ameerracle/miniconda3/envs/gpaw/bin/p...>